<a href="https://colab.research.google.com/github/caoscott/waveone/blob/master/waveone_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi
# %load_ext tensorboard
# %tensorboard --logdir .

Tue Nov 12 02:23:13 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.50       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0    88W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip

--2019-11-12 02:23:15--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.71.61.108, 52.20.12.96, 3.82.169.244, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.71.61.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.14’

ngrok-stable-linux- 100%[===================>]  13.13M   440KB/s    in 18s     

2019-11-12 02:23:34 (735 KB/s) - ‘ngrok-stable-linux-amd64.zip.14’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [0]:
get_ipython().system_raw(
    'tensorboard --logdir . --host 0.0.0.0 --port 6006 &'
)

In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

In [5]:
!curl -s http://localhost:4040/api/tunnels | python3 -c "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://76303530.ngrok.io


In [6]:
!curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id=1ddmjU48TehTk28903cg2mSoLBgkeAcwY" > /dev/null
!curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=`awk '/download/ {print $NF}' ./cookie`&id=1ddmjU48TehTk28903cg2mSoLBgkeAcwY" -o vcii_demo_data.tar.gz > /dev/null

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0   1596      0 --:--:-- --:--:-- --:--:--  1596
100  104M    0  104M    0     0  89.0M      0 --:--:--  0:00:01 --:--:--  197M


In [0]:
!tar xvf vcii_demo_data.tar.gz &> /dev/null

In [8]:
!rm -rf waveone
!git clone https://github.com/caoscott/waveone.git

Cloning into 'waveone'...
remote: Enumerating objects: 97, done.
remote: Counting objects: 100% (97/97), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 688 (delta 59), reused 67 (delta 30), pack-reused 591
Receiving objects: 100% (688/688), 113.53 KiB | 334.00 KiB/s, done.
Resolving deltas: 100% (454/454), done.


In [0]:
# !cd waveone && python3 -u train.py \
#   --train ../data/train \
#   --eval ../data/eval \
#   --save-out-img \
#   --lr 0.001 \
#   --batch-size 64 \
#   --eval-batch-size 1 \
#   --patch 64 \
#   --bits 1024 \
#   --binarize-off
from waveone.train_options import parser
from waveone.train import train

nets = None
args = parser.parse_args([
    "--train", "data/train",
    "--eval", "data/eval",
    "--lr", "0.001",
    "--batch-size", "16",
    "--eval-batch-size", "1",
    "--patch", "128",
    "--bits", "2048",
    "--binarize-off",
    "--max-train-epochs", "100",
    "--save-out-img",
])
print(args)
nets = train(args)

Namespace(batch_size=16, binarize_off=True, bits=2048, checkpoint_epochs=100, clip=0.5, eval='data/eval', eval_batch_size=1, eval_epochs=10, gamma=0.5, gpus='0', load_iter=None, load_model_name=None, lr=0.001, max_train_epochs=100, model_dir='model', out_dir='output', patch=128, save_codes=False, save_model_name='demo', save_out_img=True, schedule='50000,60000,70000,80000,90000', train='data/train', weight_decay=0.0005)
679 images loaded.
Loader for 677 images (42 batches) created.
13 images loaded.
Loader for 13 images (13 batches) created.


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2751: UserWarning: Default grid_sample and affine_grid behavior will be changed to align_corners=False from 1.4.0. See the documentation of grid_sample for details.
  warnings.warn("Default grid_sample and affine_grid behavior will be changed "
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2693: UserWarning: Default grid_sample and affine_grid behavior will be changed to align_corners=False from 1.4.0. See the documentation of grid_sample for details.
  warnings.warn("Default grid_sample and affine_grid behavior will be changed "


In [0]:
from waveone.dataset import get_loader

eval_loader = get_loader(
    is_train=False,
    root=args.eval,
    frame_len=1,
    sampling_range=0,
    args=args,
)

In [0]:
import torch
import torch.nn as nn

from waveone.losses import MSSSIM

msssim_fn = MSSSIM(val_range=1, normalize=True).cuda()
# charbonnier_loss_fn = CharbonnierLoss().cuda()
l1_loss_fn = nn.L1Loss(reduction="mean").cuda()

def eval_scores(
    frames1: torch.Tensor,
    frames2: torch.Tensor,
    prefix: str
):
    assert len(frames1) == len(frames2)
    frame_len = len(frames1)
    msssim = 0.
    l1 = 0.
    for frame1, frame2 in zip(frames1, frames2):
        l1 += l1_loss_fn(frame1, frame2)
        msssim += msssim_fn(frame1, frame2)
    return {f"{prefix}_l1": l1/frame_len,
            f"{prefix}_msssim": msssim/frame_len}

In [0]:
from collections import defaultdict
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

context_vec_test_shape = (args.eval_batch_size, 512, 144, 176)

for net in nets:
    net.eval()

encoder, binarizer, decoder = nets

with torch.no_grad():
    context_vec = torch.zeros(context_vec_test_shape).cuda()
    total_scores = defaultdict(float)
    frame1 = None

    for frame2, in eval_loader:
        frame2 = frame2.cuda()
        if frame1 is None:
            frame1 = frame2
        else:
            codes = binarizer(encoder(frame1, frame2, context_vec))
            flows, residuals, context_vec = decoder((codes, context_vec))
            flow_frame2 = F.grid_sample(frame1, flows)
            reconstructed_frame2 = (
                flow_frame2 + residuals).clamp(-0.5, 0.5)

            plt.figure()
            plt.imshow((frame2 + 0.5).cpu().squeeze().permute(1, 2, 0))
            plt.figure()
            plt.imshow((reconstructed_frame2 + 0.5).cpu().squeeze().permute(1, 2, 0))

            total_scores = {
                **eval_scores([frame1], [frame2], "eval_baseline"),
                **eval_scores([frame2], [flow_frame2], "eval_flow"),
                **eval_scores([frame2], [reconstructed_frame2], "eval_reconstructed"),
            }

            # Update frame1.
            frame1 = reconstructed_frame2

    total_scores = {k: v.item()/len(eval_loader.dataset)
                    for k, v in total_scores.items()}
    print(total_scores)

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline

img = plt.imread("pytorch-vcii/output/iter100/images/silent_cif_0012.png_iter10.png")
plt.imshow(img)

In [0]:
# def _generate_indices(index, length, times):
#     indices = []
#     for _ in range(times):
#         indices.append(index % length)
#         index //= length
#     return tuple(indices)

In [0]:
# length = 3
# times = 4
# l = set([_generate_indices(i, length, times) for i in range(length ** times)])
# len(l)

In [0]:
# from google.colab import drive
# drive.mount('/content/drive')

In [0]:
# !cp -r waveone/ "/content/drive/My Drive/"